In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy rustworkx scipy

# Quantenannähernder Optimierungsalgorithmus

*Zeitschätzung: 22 Minutn uffm Heron-r3-Prozessor (HINWEIS: Nur 'ne Schätzung — dei tatschächliche Laafzeit gann variieren.)*
## Hindergrund
Deser Tutorial zeegt, wie de'n **Quantum Approximate Optimization Algorithm (QAOA)** — 'ne hybride (quanten-glassische) iterative Methode — innerhalbb vom Qiskit-Patterns-Rahmen implementiern gannst. Erschd löse mer 's **Maximum-Cut** (oder **Max-Cut**)-Probläm fier'n gleenen Graphn, un dann lernste, wie de's im Utility-Maßschdoab ausfiehrn gannst. Alle Hardware-Ausfiehrungen im Tutorial solltn innerhalbb vom Zeitlimit vom frei zugänglischn Open Plan laafen.

's Max-Cut-Probläm is'n Obtimierungsprobläm, was schwersch ze lösn is (genauer gesachd: 's is'n *NP-schweres* Probläm) mit vielerlei Oowendungen in Clustering, Netzwärgwissnschaft un schtatistischer Fieesig. In däm Tutorial betrachtn mer'n Graphn aus Knootn, die dursch Gäntn mitnanander verbundn sinn, un woll'n die Knootn in zwee Mengen aufdeeln, sodass die Oonzahl von Gäntn, die von däm Schnitt durschqueert wärn, maximiert wird.

![Illustratschn von'm Max-Cut-Probläm](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Vorraussetzungen
Bevorn de mit däm Tutorial oogängst, schau nach, dass de Folschndes installierd hast:
- Qiskit SDK v1.0 oder neeer, mit Unnerstietzung fier [Visualisieerung](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 oder neeer (`pip install qiskit-ibm-runtime`)

Außerdem brauchste Zugrieff uff'ne Instanz uff der [IBM Quantum Platform](/guides/cloud-setup). Beaachte, dass deser Tutorial ned im Open Plan ausgefiehrd wärn gann, weil er Workloads mit [Sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session) verweendt, was nur mit dem Premium-Plan zugelassn is.
## Einrichtung

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Deel I. QAOA im gleenen Maßschdoab
Der erschde Deel von däm Tutorial nutzt'n Max-Cut-Probläm im gleenen Maßschdoab, um die Schritte ze zeechn, mit denen mer'n Obtimierungsprobläm uff'm Quantecomputer lösd.

Um 'n bissle Hintergrund ze criegen, bevorm mer das Probläm uff'n Quantealgorithmus abbildn, gann mer bässer verstehn, wie's Max-Cut-Probläm zu 'nem glassischn kombinatorischn Obtimierungsprobläm wird — oofongt mer damit, die Minimierung von ner Funktschen $f(x)$ ze betrachtn:

$$
\min_{x\in {0, 1}^n}f(x),
$$

dabei is der Eingabewärd $x$ 'n Vektor, dassen Gombonentn jeweels 'nem Gnotn vom Graphn entsprechn. Dann schränkn mer jede von dene Gombonentn uff $0$ oder $1$ ein (was bedeit: drin oder nisch drin im Schnitt). Deser gleene Beispielfall nutzt 'n Graphn mit $n=5$ Knootn.

Mer könnte 'ne Funktschen fier 'n Knootnpaar $i,j$ schreibn, die zeecht, ob die entsprechennte Gante $(i,j)$ im Schnitt is. Zum Beispiel is die Funktschen $x_i + x_j - 2 x_i x_j$ genau dann 1, wenn entweder $x_i$ oder $x_j$ gleich 1 is (was bedeit, die Gante is im Schnitt), un sonnst null. 's Probläm, die Oonzahl von Gäntn im Schnitt ze maximiern, läsd sich formuliern als:

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

was sich als 'ne Minimierung umschreebm lässd:

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

's Minimum von $f(x)$ lieecht hier vor, wenn die Oonzahl der Gäntn, die vom Schnitt durschqueert wärn, maximal is. Wie de siehst, hat das noch nix mit Quantecomputing ze du. Mer müssn das Probläm in was umformuliern, was'n Quantecomputer vastehn gann.
Fang mit der Initialisierung von deim Probläm oo, indeem de'n Graphn mit $n=5$ Knootn erstellst.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Schritt 1: Glassische Eingabn uff's Quanteprobläm abbildn
Der erschde Schritt vom Musder is es, 's glassische Probläm (Graphn) in quantemäßsche **Circuits** un **Operatorn** umzuwandln. Dafier gibt's drei Hauptschritte:

1. 'Ne Reihe von mathematischn Umformulierungen verwennd, um das Probläm in der Notatschn von Quadratic Unconstrained Binary Optimization (QUBO)-Problähmn darzustelln.
2. 's Obtimierungsprobläm als Hamiltonian umschreebm, dassen Grundsuschstand der Lösung entspricht, die die Gostenfunktschen minimiert.
3. 'Nen Quantecircuit erstell, der'n Grundsuschstand von däm Hamiltonian dursch'n Vorgang ähnlich der Quantenglüehung voorbereidt.

**Hinweis:** Bei der QAOA-Methodik willste letztlich 'nen Operator (**Hamiltonian**) habn, der die **Gostenfunktschen** von unserm hybriden Algorithmus darstellt, sowie 'nen parametrisierten Circuit (**Ansatz**), der Quantenzustände mit Kandidatenlösungen fier 's Probläm darstellt. De gannst von dene Kandidatenzuständn sampln un sie dann mit der Gostenfunktschen auswärtn.

#### Graph &rarr; Obtimierungsprobläm
Der erschde Schritt beim Abbildn is 'n Notatschnswäggsel. Das Folschnde drückt 's Probläm in QUBO-Notatschn aus:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

dabei is $Q$ 'ne $n\times n$-Matrix aus reellen Zahln, $n$ entspricht der Oonzahl von Knootn in deim Graphn, $x$ is der Vektor von binären Variabln, den mer schon kenntngelärnd habn, un $x^T$ bedeutet die Transbosiertng von Vektor $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Obtimierungsprobläm &rarr; Hamiltonian
Dann gannste 's QUBO-Probläm als **Hamiltonian** umformuliern (hier 'ne Matrix, die die Energie von'm System darstellt):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Umformulierungsschritte vom QAOA-Probläm zum Hamiltonian**
</summary>

Um ze zeechn, wie's QAOA-Probläm uff diese Weise umgeschrieebm wärn gann, tausch erschd die binären Variabln $x_i$ dursch 'ne neie Mengn von Variabln $z_i\in{-1, 1}$ aus, via:

$$
x_i = \frac{1-z_i}{2}.
$$

Hier gannste sehn, dass wenn $x_i$ gleich $0$ is, dann muss $z_i$ gleich $1$ sinn. Wenn die $x_i$'s dursch die $z_i$'s im Obtimierungsprobläm ($x^TQx$) erseztd wärn, lässd sich 'ne äquivalente Formulierung ablaeidn.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Wenn mer jetzt $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ definiern, 'n Vorfaktor wegläsn un'n gonschdantn $n^2$-Term, goommt mer bei zween äquivalentn Formulierungen von demselbn Obtimierungsprobläm oo.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Hier hängt $b$ von $Q$ ab. Beaachte, dass mer zum Erhaaldn von $z^TQz + b^Tz$ den Faktor 1/4 un 'n gonschdantn Versatz $n^2$ weggläsn haabn, die bei der Obtimierung kenne Rolle spieln.

Jetzt, um 'ne Quantenformulierung von'm Probläm ze kriengn, befördert mer die $z_i$-Variabln zu 'ner Pauli-$Z$-Matrix, etwa 'ner $2\times 2$-Matrix der Form:

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Wenn de diese Matrizen in 's Obtimierungsprobläm obn eiführst, griggte de'n Hamiltonian:

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Dazu erinnern: Die $Z$-Matrizen sinn in'n Berechnungsraum vom Quantecomputer eingebedded, also in'n Hilbert-Raum der Größe $2^n\times 2^n$. Deshalbb sollde de Ausdrieck wie $Z_iZ_j$ als Tensorprodukt $Z_i\otimes Z_j$ verstehn, das in'n $2^n\times 2^n$-Hilbert-Raum eingebedded is. Bei 'nem Probläm mit fünf Entscheeidungsvariabln wird der Ausdruck $Z_1Z_3$ zum Beispiel als $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ verstannd, wobei $I$ die $2\times 2$-Identitätsmatrix is.*
</details>

Deser Hamiltonian wird als der **Gostenfunktschn-Hamiltonian** bezäächnet. Er hat die Eigenschaft, dass sein Grundsuschstand der Lösung entspricht, die die **Gostenfunktschen $f(x)$ minimiert**.
Um dein Obtimierungsprobläm ze lösn, musste jetzt'n Grundsuschstand von $H_C$ (oder 'nen Zustand mit hoher Übberlappung damit) uff'm Quantecomputer voorbereeidn. Dann wird 's Sampln von däm Zustand mit hoher Wahrscheinlichkeet die Lösung fier $min~f(x)$ liefern.
Betrachtn mer jetzt'n Hamiltonian $H_C$ fier's **Max-Cut**-Probläm. Jedem Knootn vom Graphn is 'n Qubit im Zustand $|0\rangle$ oder $|1\rangle$ zugeordnet, wobei der Wärd die Mengn oogibt, der Gnotn zugehört. 's Ziel vom Probläm is es, die Oonzahl der Gäntn $(v_1, v_2)$ ze maximiern, bei denen $v_1 = |0\rangle$ un $v_2 = |1\rangle$ gilt, oder umgekährd. Wenn mer'n $Z$-Operator jedem Qubit zuordnet, wobei:

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

dann gehört 'ne Gante $(v_1, v_2)$ genau dann zum Schnitt, wenn der Eigenwärd von $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$ is; mit annern Wortn, die Qubits von $v_1$ un $v_2$ sinn verschieden. Genauso gehört $(v_1, v_2)$ ned zum Schnitt, wenn der Eigenwärd von $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$ is. Beaachte, dass's uns ned uff'n genaun Quantezustand von jedem Gnotn ookommt, sunnern nur darauf, ob se übbr 'ne Gante gleich oder verschieden sinn. 's Max-Cut-Probläm ferderd, 'ne Zuweisung von Qubits uff Knootn ze findn, sodass der Eigenwärd von folschndem Hamiltonian minimiert wird:
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Mit annern Wortn: $b_i = 0$ fier alle $i$ beim Max-Cut-Probläm. Der Wärd $Q_{ij}$ gibt's Gewicht von der Gante oo. In däm Tutorial betrachtn mer'n ungewichttn Graphn, also $Q_{ij} = 1.0$ fier alle $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Schritt 2: Probläm fier die Quantehardwareausfiehrung obtimieern
Der Circuit obn enthält 'ne Reihe von Abstraktschonen, die nitzlich is, um übbr Quantealgorithmn nachzedenggn, aber ned uff der Hardware ausgefiehrd wärn gann. Um uff'm QPU laafen ze gönnen, muss der Circuit 'ne Reihe von Oberatschonen durchlaafen, die den **Transpilatschn**- oder **Circuit-Obtimierung**-Schritt vom Muster ausmachn.

Die Qiskit-Bibliotheg bietet 'ne Reihe von **Transpilatschnspässen**, die 'ne breitere Palette von Circuit-Transformatschn abdeckn. De musst sicherstelln, dass dein Circuit fier deinen Zweck **obtimiert** is.

Die Transpilatschn gann meehrere Schritte umfassn, wie:

* **Initiale Abbildung** der Qubits im Circuit (zum Beispiel Entscheeidungsvariabln) uff physische Qubits uff'm Gerät.
* **Entfaltung** der Instruktschonen im Quantencircuit in die hardware-nativen Instruktschonen, die das Backend vastehd.
* **Routing** von Qubits im Circuit, die mitnanander interagieern, zu physischn Qubits, die nebenanandr liengn.
* **Fählerunnerdrickung** dursch Hinzufiengn von Single-Qubit-Gates zur Rauschunnerdrickung mit dynamischem Entkoppln.

Mehr Informatschn iebr Transpilatschn finddste in unsern [Dokus](/guides/transpile).

Der folschnde Code transformiert un obtimiert'n abstraktn Circuit in 'n Format, das bereit zur Ausfiehrung uff einem der übbr die Cloud zugänglischn Geräte is — mit dem **Qiskit IBM Runtime-Dienscht**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Schritt 3: Ausfiehrung mit Qiskit-Primitiven
Im QAOA-Workflow wärn die obtimalen QAOA-Parameter in 'ner iterativen Obtimierungsschleife gefundn, die 'ne Reihe von Circuit-Auswärtngen ausfiehrd un 'nen glassischn Optimierer nutzd, um die obtimalen $\beta_k$- un $\gamma_k$-Parameter ze findn. Diese Ausfiehrungsschleife wird in folschnden Schrittn ausgefiehrd:

1. Die Anfangsparameter definiern
2. 'Ne neie `Session` instantiiern, die die Obtimierungsschleife un 's Primitive zum Sampln vom Circuit enthält
3. Sobald 'n optimaler Parametersatz gefundn wurrd, führt mer'n Circuit noch mal aus, um 'ne endgültige Värdeilung ze criengn, die im Noochbarbeeitungsschritt verweendd wird.
#### Circuit mit Anfangsparametern definiern
Mer starn mit willkürlich gewählten Parametern.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Backend un Ausfiehrungsprimitiv definiern
Nutz die **Qiskit Runtime-Primitive**, um mit IBM&reg;-Backends ze interagieern. Die zwee Primitive sinn Sampler un Estimator, un die Wahl vom Primitiv hängt davon ab, welche Art von Mässung de uff'm Quantecomputer machen willst. Fier die Minimierung von $H_C$ nutzde'n Estimator, weil die Mässung der Gostenfunktschen eenfach dem Erwartungswärd $\langle H_C \rangle$ entspricht.
#### Ausfiehrn
Die Primitive bieten vielerlei [Ausfiehrungsmodi](/guides/execution-modes) zum Plaanen von Workloads uff Quantegerätn, un'n QAOA-Workflow läfd iterativ in 'ner Session.

![Illustratschn, die 's Verhaltn von Single-Job-, Batch- un Session-Laufzeitmodi zeeched.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

De gannst die sampler-basierte Gostenfunktschen in die SciPy-Minimierungsroutine eieplugn, um die obtimalen Parameter ze findn.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Sobald de die obtimalen Parameter fier'n Circuit gefundn hast, gannste diese Parameter zuweisn un die endgültige Värdeilung sampln, die mit den obtimiertn Parametern entstehd. Hier sollde'n *Sampler*-Primitiv genutzd wärn, weil's die Wahrscheinlichkeedsvärdeilung von Bitstring-Mässungen is, die dem obtimalen Schnitt vom Graphn entsprichn.

**Hinweis:** Das bedeit, 'nen Quantenzustand $\psi$ im Computer voorzubereeidn un ihn dann ze mässn. 'Ne Mässung kollabiert'n Zustand uff 'nen einzeln Berechnungsbasissuschstand — zum Beispiel `010101110000...` — was 'ner Kandidatenlösung $x$ fier unser Ursprungsoptimierungsprobläm entspricht ($\max f(x)$ oder $\min f(x)$, je nach Aufgabe).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Schritt 4: Noochbarbeitung un Rückgoabe vom Ergebnis im gewünschtn glassischn Format
Der Noochbarbeeitungsschritt interpretierd die Sampling-Ausgabe, um 'ne Lösung fier dein ursprünglisches Probläm zurückzegeebm. In däm Fall interessiert dich der Bitstring mit der höchstn Wahrscheinlichkeet, weil der'n obtimalen Schnitt bestimmt. Durch die Symmetrien im Probläm gibt's vier mögliche Lösngen, un 's Sampling-Verfahrn lied eine davon mit etwas höherer Wahrscheinlichkeet zurück — aber wie de in der Värdeilungsdarstellung unten siehst, sinn vier Bitstrings deutlich wahrscheinlicher als der Rest.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Besctn Schnitt visualisiern
Vom obtimalen Bitstring aus gannste dann desen Schnitt uff'm ursprünglischn Graphn visualisiern.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

Un berächne den Wärd vom Schnitt:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Deel II. Jetzt uff groß!
De hast Zugrieff uff viele Geräte mit übbr 100 Qubits uff IBM Quantum&reg; Platform. Wähl eens aus, uff dem de Max-Cut uff'm 100-Gnotn-gewichttn Graphn lösen willst. Das is'n „Utility-Scale"-Probläm. Die Schritte zum Aufbau von'm Workflow sinn dieselbn wie obn, aber mit'nem viel größern Graphn.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Schritt 1: Glassische Eingabn uff's Quanteprobläm abbildn
#### Graph &rarr; Hamiltonian
Wandel zunächst den Graphn, denn de lösn willst, direkt in 'nen Hamiltonian um, der fier QAOA geeignet is.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamiltonian &rarr; Quantencircuit

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Ausgabe der vorherigen Code-Zälle](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Schritt 2: Probläm fier die Quanteausfiehrung obtimieern
Um den Circuit-Obtimierungsschritt uff Utility-Scale-Problähmn hochzusgaliern, gannste die hoochleistungsfähigen Transpilatschnssdrategien nutzn, die mit Qiskit SDK v1.0 eingefiert wordn sinn. Weitere Tools sinn der neue Transpilerdienst mit [KI-verbässerten Transpilatschnspässn](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Sobald die obtimalen Parameter vom Ooläfn von QAOA uff'm Gerät gefundn wordn sinn, weise die Parameter dem Circuit zu.